In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('health_data.csv')
data.head()

,age,restbps,chol,category
0,26,109,243,0
1,27,106,156,0
2,28,107,225,0
3,27,105,277,0
4,30,96,221,0


In [5]:
X = data.to_numpy()[:, :3]
Y = data.to_numpy()[:, 3]


split = 0.7 #using 70:30 split, random each time
train_samples = np.random.choice(X.shape[0], int(split*X.shape[0]))
test_samples = np.array([i for i in range(X.shape[0]) if i not in list(train_samples)])

X_train = X[[train_samples]]
Y_train = Y[[train_samples]]

X_test = X[[test_samples]]
Y_test = Y[[test_samples]]


X_0 = X_train[Y_train == 0]
X_1 = X_train[Y_train == 1]

#X_0 = data[data['category'] == 0].to_numpy()[:, :3]
#X_1 = data[data['category'] == 1].to_numpy()[:, :3]
#how to split data into test and train?

C:\Users\Acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if __name__ == '__main__':
C:\Users\Acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted

In [8]:
linear_svm = SVC(C=1.0, kernel='linear', random_state=42)
linear_svm.fit(X_train, Y_train)
#that was pretty fast

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=42,
    shrinking=True, tol=0.001, verbose=False)

In [10]:
linear_svm.decision_function(X_train)

array([-1.78482891e+00,  2.19381935e+00,  4.52395938e+00, -2.79699288e+00,
       -1.32430258e+00, -2.49761262e+00, -8.04342302e-01, -2.70664071e+00,
       -9.76985989e-01, -4.59013239e-01,  7.39774749e-02, -4.58434682e-02,
        1.00041295e+00, -2.56085680e+00,  5.10845416e-01, -8.78214442e-01,
       -3.15312857e-01, -1.80003886e+00, -9.99649661e-01, -7.25087341e-01,
       -2.18877848e+00,  2.15224602e+00,  3.36294183e-03, -1.30706341e+00,
        2.52465591e+00, -2.73667473e+00, -3.12189400e+00, -2.45893817e+00,
        2.14280477e+00, -1.40627706e+00,  4.48937052e+00,  1.14119992e+00,
       -2.03391205e+00, -2.63602617e+00, -6.99151532e-01, -6.52443590e-01,
       -1.42333686e+00,  2.07312874e+00, -2.27692207e+00, -4.21773700e-01,
       -2.16295320e+00,  1.50471236e-01, -2.78087158e+00,  3.53131201e+00,
        1.96676579e+00, -2.92223832e+00,  8.13952536e-01, -6.00490509e-01,
        1.26681863e+00, -2.08492663e+00, -1.68481444e+00,  8.65035952e-01,
        1.42238798e+00,  

In [12]:
Y_pred = linear_svm.predict(X_train)
conf = np.zeros((2,2))

for y, y_ in zip(Y_train, Y_pred):
    conf[y][y_] += 1

print(conf)

[[239.  30.]
 [ 42. 178.]]


In [13]:
Y_pred = linear_svm.predict(X_test)
conf = np.zeros((2,2))

for y, y_ in zip(Y_train, Y_pred):
    conf[y][y_] += 1

print(conf)

[[115.  75.]
 [ 72.  82.]]
